In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Importing dataset

In [ ]:
df=pd.read_csv('../input/house-rent-prediction-dataset/House_Rent_Dataset.csv')

In [ ]:
df.head()

In [ ]:
num_c=list(df.select_dtypes([np.number]).columns)

In [ ]:
num_c

In [ ]:
cat_col=list(set(df.columns)-set(num_c))

In [ ]:
cat_col

In [ ]:
#df['Floor'].unique()

In [ ]:
df['Floor']=df['Floor'].map(lambda x:x.strip('out')[0])

In [ ]:
df['Floor'].unique()

In [ ]:
df['Floor']=df['Floor'].replace(['G'],0)
df['Floor']=df['Floor'].replace(['U'],-1)
df['Floor']=df['Floor'].replace(['L'],-2)

In [ ]:
df['Floor'].unique()

In [ ]:
df['year']=pd.DatetimeIndex(df['Posted On']).year

In [ ]:
df['month']=pd.DatetimeIndex(df['Posted On']).month

In [ ]:
df['month'].value_counts()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['Floor']=df['Floor'].astype(int)

In [ ]:
df.corr()

In [ ]:
sns.heatmap(df.corr(),annot=True)

In [ ]:
df['Tenant Preferred'].unique()

# checking for null values

In [ ]:
df.isnull().sum()

In [ ]:
df['City'].unique()

In [ ]:
df['Point of Contact'].unique()

In [ ]:
df.groupby('Point of Contact')['Rent'].mean()

In [ ]:
#df.groupby(['City','Point of Contact','BHK'])['Rent'].mean().round()

In [ ]:
#df.groupby(['City','BHK'])['Rent'].max().round()

# EDA

**Univariant Analysis**

In [ ]:
plt.figure(figsize=(12,8))
sns.lineplot(data=df,x=df['month'],y='Rent')

In [ ]:
sns.barplot(data=df,x=df['month'],y='Rent')

There is trend in increase of rent with time( may be beacuse of demand)

In [ ]:
sns.barplot(data=df,x=df['City'],y=df['Rent'])

> Rent is costly in Mumbai(on an average)

In [ ]:
sns.countplot(data=df,x=df['BHK'])

>  2 BHK type houses are more compared to other

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(data=df,x=df['City'],y=df['Rent'],hue=df['BHK'])

> Rent of house is increasing with increase in BHKs

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(data=df,x=df['City'],y=df['Rent'],hue=df['Bathroom'])

**** I want to see the house with 10 bathrooms ****

In [ ]:
sns.countplot(data=df,x=df['Furnishing Status'])

In [ ]:
sns.barplot(data=df,x=df['City'],y=df['Rent'],hue=df['Furnishing Status'])

> Furnished houses are costly (most places)

In [ ]:
sns.countplot(data=df,x=df['Tenant Preferred'])

In [ ]:
sns.barplot(data=df,x=df['City'],y=df['Rent'],hue=df['Tenant Preferred'])

In [ ]:
sns.countplot(data=df,x='Area Type')

In [ ]:
sns.barplot(data=df,x='City',y='Rent',hue='Area Type')

> Rent in carpet area is high

In [ ]:
df[df['Rent']==df['Rent'].max()]

> Maximum house rent is in banglore (marathahalli locality)

In [ ]:

sns.barplot(data=df,x=df['Point of Contact'],y=df['Rent'],hue=df['City'])

>  Its better to contact owner directly
>  and We can contact builder (much cheaper option) in hyderabad 

In [ ]:
sns.regplot(data=df,y=df['Rent'],x=df['Size'])

> There is clear correlation between size of house and rent

In [ ]:
df.drop(columns=['Posted On','Area Locality'],inplace=True)

In [ ]:
sns.countplot(x=df['Floor'])

In [ ]:
sns.barplot(data=df,x='Floor',y='Rent')

In [ ]:
df.head()

# Encoding categorical variables

In [ ]:
#from sklearn.preprocessing import LabelEncoder
#le=LabelEncoder()
#le.fit(df['Area Type'])
#df['Area Type']=le.transform(df['Area Type'])
#le.fit(df['City'])
#df['City']=le.transform(df['City'])
#le.fit(df['Furnishing Status'])
#df['Furnishing Status']=le.transform(df['Furnishing Status'])
#le.fit(df['Tenant Preferred'])
#df['Tenant Preferred']=le.transform(df['Tenant Preferred'])
#le.fit(df['Point of Contact'])
#df['Point of Contact']=le.transform(df['Point of Contact'])

In [ ]:
en=pd.get_dummies(data=df[['Area Type','City','Furnishing Status','Tenant Preferred','Point of Contact']],drop_first=True)

In [ ]:
df.drop(columns=['Area Type','City','Furnishing Status','Tenant Preferred','Point of Contact'],inplace=True)

In [ ]:
df=pd.concat([en,df],axis=1)

In [ ]:
X=df.drop(columns='Rent')
y=df['Rent']

# SPLITTING THE DATA

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

# Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train[['Size']]=sc.fit_transform(X_train[['Size']])
X_test[['Size']]=sc.transform(X_test[['Size']])
X_train[['year']]=sc.fit_transform(X_train[['year']])
X_test[['year']]=sc.transform(X_test[['year']])

In [ ]:
X_train

# Fitting with model

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,y_train)

In [ ]:
yp=lr.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_absolute_percentage_error
print("r2_score :",r2_score(yp,y_test))
print(mean_absolute_error(yp,y_test))
mean_absolute_percentage_error(yp,y_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(X_train,y_train)
ypr=rf.predict(X_test)
print("r2_score :",r2_score(ypr,y_test))
print(mean_absolute_error(ypr,y_test))
mean_absolute_percentage_error(ypr,y_test)

**Random forest is the best model for our dataset**

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dt=DecisionTreeRegressor()
dt.fit(X_train,y_train)
ypd=dt.predict(X_test)
print("r2_score :",r2_score(ypd,y_test))
print(mean_absolute_error(ypd,y_test))
mean_absolute_percentage_error(ypd,y_test)

In [ ]:
from sklearn.linear_model import Lasso
l=Lasso()
l.fit(X_train,y_train)
lp=l.predict(X_test)
print("r2_score :",r2_score(lp,y_test))
print(mean_absolute_error(lp,y_test))
mean_absolute_percentage_error(lp,y_test)